In [1]:
import polars as pl
import pandas as pd

# 讀取 parquet
train_filled = pl.read_parquet("data/train_filled.parquet")

# 讀取top50 features
n_top = 20
save_dir = "model_output/selected_features_xgb/one_model"
df = pd.read_csv("model_output/all_features_xgb/model_importance/common_features_with_ranks.csv")
top50_features = df["feature"].head(n_top).tolist()

# 你要保留的欄位 (Top50 + target + group id)
cols_to_keep = top50_features + ["selected", "ranker_id"]

# 只保留這些欄位
train_filled = train_filled.select(cols_to_keep)

print(f"✅ DataFrame 現在只有 {len(train_filled.columns)} 欄位: {train_filled.columns}")


✅ DataFrame 現在只有 22 欄位: ['total_num_transfers_rank', 'total_is_min_transfers', 'legs0_num_transfers_rank', 'price_from_median_zscore', 'price_percentile', 'legs0_segments0_flightNumber', 'price_per_duration', 'price_per_duration_rank', 'companyID', 'has_access_tp', 'pricingInfo_isAccessTP', 'days_before_departure', 'legs0_mean_cabin', 'price_minus_fee_rank', 'price_per_fee', 'price_per_fee_rank', 'both_legs_carrier_all_same', 'legs0_arrivalAt_hour', 'legs0_departureAt_hour', 'price_per_tax', 'selected', 'ranker_id']


In [2]:
exclude_cols = [
    'Id', 'ranker_id', 'selected'
]

feature_cols = [col for col in train_filled.columns if col not in exclude_cols]

print(f"Using {len(feature_cols)} features")

X = train_filled.select(feature_cols)
y = train_filled.select('selected')
groups = train_filled.select('ranker_id')

Using 20 features


In [3]:
import numpy as np
import polars as pl

# 確認這些物件都是Polars DataFrame
# X, y, groups
# 都是 shape [n_rows, n_cols]

# 先把 ranker_id轉list
unique_rankers = groups.select("ranker_id").unique().to_series().to_list()

# 打亂
np.random.seed(42)
np.random.shuffle(unique_rankers)

# 切8:2
n_train = int(0.8 * len(unique_rankers))
train_rankers = set(unique_rankers[:n_train])
val_rankers = set(unique_rankers[n_train:])

# 用 Polars 過濾 train/val
is_train = groups.select(pl.col("ranker_id").is_in(list(train_rankers)).alias("is_train"))

# 先 concat mask
X_with_mask = X.with_columns(is_train)
y_with_mask = y.with_columns(is_train)
groups_with_mask = groups.with_columns(is_train)

# 分割 DataFrame
X_train_df = X_with_mask.filter(pl.col("is_train"))
X_val_df = X_with_mask.filter(~pl.col("is_train"))
y_train_df = y_with_mask.filter(pl.col("is_train"))
y_val_df = y_with_mask.filter(~pl.col("is_train"))
groups_train_df = groups_with_mask.filter(pl.col("is_train"))
groups_val_df = groups_with_mask.filter(~pl.col("is_train"))

# 再轉 numpy (分批)
X_train_np = X_train_df.drop("is_train").to_numpy()
X_val_np = X_val_df.drop("is_train").to_numpy()
y_train_np = y_train_df.drop("is_train").to_numpy().flatten()
y_val_np = y_val_df.drop("is_train").to_numpy().flatten()
groups_train_np = groups_train_df.drop("is_train").to_numpy().flatten()
groups_val_np = groups_val_df.drop("is_train").to_numpy().flatten()
# 最後計算 group sizes
group_sizes_train = (
    groups_train_df.drop("is_train")
    .group_by("ranker_id", maintain_order=True)
    .agg(pl.len())['len']
    .to_numpy()
)

group_sizes_val = (
    groups_val_df.drop("is_train")
    .group_by("ranker_id", maintain_order=True)
    .agg(pl.len())['len']
    .to_numpy()
)


In [ ]:
import lightgbm as lgb
import os

# 📌 參數
lgb_params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "ndcg_eval_at": [3],
    "max_depth": 10,
    "min_data_in_leaf": 10,      # ≈ min_child_weight
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "lambda_l2": 10.0,           # XGB 的 lambda
    "learning_rate": 0.05,
    "seed": 42,
    "num_threads": -1,
    "verbosity": 20
}

# 📌 建立 Dataset
dtrain = lgb.Dataset(
    X_train_np,
    label=y_train_np,
    feature_name = feature_cols,
    group=group_sizes_train
)
dval = lgb.Dataset(
    X_val_np,
    label=y_val_np,
    feature_name = feature_cols,
    group=group_sizes_val,
    reference=dtrain
)

# 📌 訓練
lgb_model = lgb.train(
    lgb_params,
    dtrain,
    num_boost_round=500,
    valid_sets=[dtrain, dval],
    valid_names=["train", "val"],
)

# 📌 儲存模型
model_path = os.path.join(save_dir, f"top{n_top}")
os.makedirs(model_path, exist_ok=True)
model_file = os.path.join(model_path, "lgb_ranker_model.txt")

lgb_model.save_model(model_file)
print(f"✅ 已儲存模型到 'lgb_ranker_model.txt' in {model_file}")


[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=100) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1267650600228229401496703205376) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=100) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1267650600228229401496703205376) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.817410
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.210568
[LightGBM] [Debug] init for col-wise cost 0.217633 seconds, init for row-wise cost 0.682065 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testi

In [17]:
import polars as pl
from scripts.hitrate import compute_hitrate_at_3
# 預測
val_preds = lgb_model.predict(X_val_np)

# 計算 HitRate
compute_hitrate_at_3(groups_val_np, y_val_np, val_preds)
import pandas as pd
import os

# 三種重要性對應 LightGBM
importance_types = {
    "weight": "split",    # 等於 XGBoost 的 "weight"
    "gain": "gain",       # 等於 XGBoost 的 "gain"
    # LightGBM 沒有 "cover"，只有 split/gain
}

importance_all = {}

for k, lgb_type in importance_types.items():
    imp_raw = lgb_model.feature_importance(importance_type=lgb_type)
    imp_named = dict(zip(lgb_model.feature_name(), imp_raw))
    sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
    importance_all[k] = sorted_imp

# weight榜
df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight_rank"])
df_weight["weight_rank_pos"] = df_weight.index

# gain榜
df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain_rank"])
df_gain["gain_rank_pos"] = df_gain.index

# 合併
df_merged = (
    df_weight
    .merge(df_gain, on="feature", how="outer")
)

# 補空值
df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)

# 最小rank
df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos"]].min(axis=1)

# 排序
df_merged_sorted = df_merged.sort_values("min_rank")

# Top50
top50 = df_merged_sorted.head(50)

# 顯示
print(top50[["feature", "weight_rank", "gain_rank"]])

# 輸出CSV
csv_path = os.path.join(save_dir, f"top{n_top}/lgb_feature_importance.csv")
df_merged_sorted.to_csv(csv_path, index=False)
print(f"✅ 已輸出 feature_importance.csv 到 {csv_path}")


✅ HitRate@3 (groups size in [10, inf]): 0.4376
                         feature  weight_rank      gain_rank
13                 price_per_fee         1779  162549.558062
18        total_is_min_transfers           73  484589.317835
19      total_num_transfers_rank          103  412727.427146
11            price_per_duration         1572  132229.592556
8   legs0_segments0_flightNumber         1487   86169.376355
15                 price_per_tax         1469   77070.443410
9       price_from_median_zscore          886  121969.296433
1                      companyID         1321   55707.555080
16              price_percentile         1122  105525.248924
12       price_per_duration_rank          888   67554.778963
4           legs0_arrivalAt_hour          786   43623.019606
5         legs0_departureAt_hour          766   34222.141737
14            price_per_fee_rank          667   42109.144300
2          days_before_departure          657   21710.026397
10          price_minus_fee_rank      

In [4]:
import xgboost as xgb

# DMatrix 建立 (不再用 X.columns)
dtrain = xgb.DMatrix(
    X_train_np,
    label=y_train_np,
    feature_names = feature_cols,
)
dtrain.set_group(group_sizes_train)

dval = xgb.DMatrix(
    X_val_np,
    label=y_val_np,
    feature_names = feature_cols,

)
dval.set_group(group_sizes_val)


In [ ]:

# 參數
xgb_params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    'max_depth': 10,
    'min_child_weight': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 10.0,
    'learning_rate': 0.05,
    'seed': 42,
    'n_jobs': -1,
}

# 訓練
xgb_model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=500,
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=50,
    verbose_eval=20
)
# 存成二進位模型
import os
model_path = os.path.join(save_dir, f"top{n_top}")
os.makedirs(model_path, exist_ok=True)
model_path = os.path.join(save_dir, f"top{n_top}/xgb_ranker_model.bin")

xgb_model.save_model(model_path)
print(f"✅ 已儲存模型到 'xgb_ranker_model.bin' in {model_path}")



[0]	train-ndcg@3:0.32043	val-ndcg@3:0.31948
[20]	train-ndcg@3:0.40374	val-ndcg@3:0.37549
[40]	train-ndcg@3:0.42629	val-ndcg@3:0.38692
[60]	train-ndcg@3:0.44314	val-ndcg@3:0.39480
[80]	train-ndcg@3:0.45610	val-ndcg@3:0.40109
[100]	train-ndcg@3:0.46769	val-ndcg@3:0.40548
[120]	train-ndcg@3:0.47658	val-ndcg@3:0.40964
[140]	train-ndcg@3:0.48752	val-ndcg@3:0.41397
[160]	train-ndcg@3:0.49736	val-ndcg@3:0.41775
[180]	train-ndcg@3:0.50615	val-ndcg@3:0.42066
[200]	train-ndcg@3:0.51277	val-ndcg@3:0.42354
[220]	train-ndcg@3:0.52169	val-ndcg@3:0.42562
[240]	train-ndcg@3:0.53101	val-ndcg@3:0.42851
[260]	train-ndcg@3:0.54018	val-ndcg@3:0.43037
[280]	train-ndcg@3:0.55078	val-ndcg@3:0.43334
[300]	train-ndcg@3:0.55996	val-ndcg@3:0.43632
[320]	train-ndcg@3:0.56919	val-ndcg@3:0.43836
[340]	train-ndcg@3:0.57731	val-ndcg@3:0.44158
[360]	train-ndcg@3:0.58453	val-ndcg@3:0.44314
[380]	train-ndcg@3:0.59107	val-ndcg@3:0.44476
[400]	train-ndcg@3:0.59701	val-ndcg@3:0.44582
[420]	train-ndcg@3:0.60255	val-ndcg@3:0.

XGBoostError: [09:29:52] /Users/runner/miniforge3/conda-bld/xgboost-split_1748292887431/work/dmlc-core/src/io/local_filesys.cc:210: Check failed: allow_null:  LocalFileSystem::Open "model_output/selected_features_xgb/one_model/top20/xgb_ranker_model.bin": No such file or directory
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000bae0fe0f0 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x0000000bae4c4b18 dmlc::io::LocalFileSystem::Open(dmlc::io::URI const&, char const*, bool) + 752
  [bt] (2) 3   libxgboost.dylib                    0x0000000bae4b9828 dmlc::Stream::Create(char const*, char const*, bool) + 68
  [bt] (3) 4   libxgboost.dylib                    0x0000000bae183598 XGBoosterSaveModel + 120
  [bt] (4) 5   libffi.8.dylib                      0x0000000105408050 ffi_call_SYSV + 80
  [bt] (5) 6   libffi.8.dylib                      0x0000000105405868 ffi_call_int + 1452
  [bt] (6) 7   _ctypes.cpython-311-d

In [ ]:
import polars as pl
from scripts.hitrate import compute_hitrate_at_3
# 預測
val_preds = xgb_model.predict(dval)

# 計算 HitRate
compute_hitrate_at_3(groups_val_np, y_val_np, val_preds)


TypeError: Cannot use Dataset instance for prediction, please use raw data instead

In [8]:
# 取得重要性 (index key: f0, f1, ...)
importance_dict = xgb_model.get_score(importance_type="weight")

# 把特徵名稱對應回 X.columns
# 如果X是 Polars DataFrame
feature_names = list(X.columns)

# 重新命名
importance_named = {}
for k, v in importance_dict.items():
    # k = 'f0', 'f1', ...
    importance_named[k] = v

# 排序
sorted_importance = sorted(importance_named.items(), key=lambda x: x[1], reverse=True)

# 列印
print("⭐ Top 20 Feature Importances (by weight):")
for feat, score in sorted_importance[:20]:
    print(f"{feat}: {score}")


⭐ Top 20 Feature Importances (by weight):
price_per_duration: 21575.0
price_per_tax: 20955.0
price_per_fee: 20054.0
legs0_segments0_flightNumber: 19662.0
companyID: 18926.0
price_per_duration_rank: 18690.0
price_from_median_zscore: 17851.0
price_percentile: 16478.0
days_before_departure: 14933.0
price_per_fee_rank: 14852.0
price_minus_fee_rank: 14395.0
legs0_arrivalAt_hour: 13306.0
legs0_departureAt_hour: 12719.0
has_access_tp: 1591.0
both_legs_carrier_all_same: 1226.0
legs0_mean_cabin: 1101.0
total_num_transfers_rank: 512.0
legs0_num_transfers_rank: 403.0
pricingInfo_isAccessTP: 288.0
total_is_min_transfers: 53.0


In [ ]:
import pandas as pd

# 先準備特徵名稱
feature_names = list(X.columns)

# 取三種重要性
importance_types = ["weight", "gain", "cover"]
importance_all = {}

for imp_type in importance_types:
    imp_raw = xgb_model.get_score(importance_type=imp_type)
    imp_named = {}
    for k, v in imp_raw.items():
        imp_named[k] = v
    # 排序
    sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
    importance_all[imp_type] = sorted_imp

# 把三個榜單放成DataFrame方便比對
df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight_rank"])
df_weight["weight_rank_pos"] = df_weight.index

df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain_rank"])
df_gain["gain_rank_pos"] = df_gain.index

df_cover = pd.DataFrame(importance_all["cover"], columns=["feature", "cover_rank"])
df_cover["cover_rank_pos"] = df_cover.index

# 合併
df_merged = (
    df_weight
    .merge(df_gain, on="feature", how="outer")
    .merge(df_cover, on="feature", how="outer")
)

# 把不存在的rank補大數字
df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)
df_merged["cover_rank_pos"] = df_merged["cover_rank_pos"].fillna(9999)

# 計算「三個榜單中最早出現的位置」
df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos", "cover_rank_pos"]].min(axis=1)

# 排序
df_merged_sorted = df_merged.sort_values("min_rank")

# 取前50
top50 = df_merged_sorted.head(50)

# 顯示
print(top50[["feature", "weight_rank", "gain_rank", "cover_rank"]])
# 如果想輸出CSV
csv_path = os.path.join(save_dir, f"top{n_top}/feature_importance.csv")

df_merged_sorted.to_csv(csv_path, index=False)
print("✅ 已輸出 feature_importance_top50.csv")


AttributeError: 'Booster' object has no attribute 'get_score'

# Prediction

In [2]:
import os
import numpy as np
import polars as pl
import lightgbm as lgb

# 參數
model_path = "model_output/selected_features_xgb/one_model/top20/lgb_ranker_model.txt"
parquet_path = "data/test_filled.parquet"

# 讀取 LightGBM 模型
lgb_model = lgb.Booster(model_file=model_path)

# 取得 feature_names
model_features = lgb_model.feature_name()
if model_features is None or len(model_features) == 0:
    raise ValueError("❌ 模型沒有 feature_names，請確認訓練時有指定 feature_names")
print(f"✅ 模型共 {len(model_features)} 個features")

# 讀取 test_filled
df = pl.read_parquet(parquet_path)
print(f"✅ 讀取 test_filled，共 {df.height} rows")

# 檢查缺失
missing_in_data = [f for f in model_features if f not in df.columns]
if missing_in_data:
    raise ValueError(f"❌ 下列特徵在 test_filled 不存在: {missing_in_data}")

# 篩選&排序
df_for_predict = df.select(model_features)
X_np = df_for_predict.to_numpy()

# 預測
preds = lgb_model.predict(X_np)
print(f"✅ 預測完成，共 {len(preds)} 筆")

# 回存結果
df_result = (
    df
    .with_columns([
        pl.Series("selected", preds)
    ])
)

# 查看前幾筆
print(df_result.head())


✅ 模型共 20 個features
✅ 讀取 test_filled，共 6897776 rows
✅ 預測完成，共 6897776 筆
shape: (5, 236)
┌──────────┬────────┬───────────┬────────────┬───┬────────────┬────────────┬────────────┬──────────┐
│ Id       ┆ bySelf ┆ companyID ┆ nationalit ┆ … ┆ legs1_segm ┆ legs1_segm ┆ searchRout ┆ selected │
│ ---      ┆ ---    ┆ ---       ┆ y          ┆   ┆ ents2_flig ┆ ents3_flig ┆ e          ┆ ---      │
│ i64      ┆ i8     ┆ i64       ┆ ---        ┆   ┆ htNumber   ┆ htNumber   ┆ ---        ┆ f64      │
│          ┆        ┆           ┆ i64        ┆   ┆ ---        ┆ ---        ┆ i32        ┆          │
│          ┆        ┆           ┆            ┆   ┆ i32        ┆ i32        ┆            ┆          │
╞══════════╪════════╪═══════════╪════════════╪═══╪════════════╪════════════╪════════════╪══════════╡
│ 18144679 ┆ 1      ┆ 62840     ┆ 36         ┆ … ┆ 8790       ┆ -1         ┆ 3236       ┆ 0.576973 │
│ 18144680 ┆ 1      ┆ 62840     ┆ 36         ┆ … ┆ 8790       ┆ -1         ┆ 3236       ┆ 0.500844 │
│ 181

In [ ]:
import os
import numpy as np
import polars as pl
import xgboost as xgb

# 參數
model_path = "model_output/selected_features_xgb/one_model/top20/xgb_ranker_model.bin"
parquet_path = "data/test_filled.parquet"

# 讀取模型
xgb_model = xgb.Booster(model_file=model_path)

# 確認 feature_names
model_features = xgb_model.feature_names
if model_features is None:
    raise ValueError("❌ 模型沒有 feature_names，請確認訓練時有指定 feature_names")
print(f"✅ 模型共 {len(model_features)} 個features")

# 讀取 test_filled
df = pl.read_parquet(parquet_path)
print(f"✅ 讀取 test_filled，共 {df.height} rows")

# 檢查缺失
missing_in_data = [f for f in model_features if f not in df.columns]
if missing_in_data:
    raise ValueError(f"❌ 下列特徵在 test_filled 不存在: {missing_in_data}")

# 篩選&排序
df_for_predict = df.select(model_features)
X_np = df_for_predict.to_numpy()

# 預測
dtest = xgb.DMatrix(X_np, feature_names=model_features)
preds = xgb_model.predict(dtest)
print(f"✅ 預測完成，共 {len(preds)} 筆")

# 回存結果
df_result = (
    df
    .with_columns([
        pl.Series("selected", preds)
    ])
)

# 查看前幾筆
print(df_result.head())


XGBoostError: std::bad_alloc

In [ ]:
import os
import polars as pl

def export_submission_parquets(
    test_filled_with_preds: pl.DataFrame,
    output_dir: str,
    raw_filename: str = "raw_submission.parquet",
    ranked_filename: str = "rank_submission.parquet"
):
    """
    根據 test_filled_with_preds 輸出兩個 parquet:
    1. 原始分數 (selected)
    2. rank 排序 (selected)
    """
    # 檢查目錄
    os.makedirs(output_dir, exist_ok=True)

    # Subset + __index_level_0__
    subset_df = (
        test_filled_with_preds
        .select(["Id", "ranker_id", "selected"])
        .with_columns(
            pl.col("Id").alias("__index_level_0__")
        )
        .with_columns([
            pl.col("Id").cast(pl.Int64),
            pl.col("ranker_id").cast(pl.Utf8),
            pl.col("selected").cast(pl.Float64),
            pl.col("__index_level_0__").cast(pl.Int64)
        ])
    )

    # 儲存原始 parquet
    raw_path = os.path.join(output_dir, raw_filename)
    subset_df.write_parquet(raw_path)
    print(f"✅ 已儲存原始 submission: {raw_path}")
    print(subset_df)

    # Rank 排名
    ranked_df = subset_df.with_columns(
        pl.col("selected")
          .rank(method="ordinal", descending=True)
          .over("ranker_id")
          .alias("selected")
    )

    # 儲存排名 parquet
    ranked_path = os.path.join(output_dir, ranked_filename)
    ranked_df.write_parquet(ranked_path)
    print(f"✅ 已儲存rank submission: {ranked_path}")
    print(ranked_df)
    
n_top = 20
export_submission_parquets(
    test_filled_with_preds=df_result,   # 你的帶有 selected 分數的 DataFrame
    output_dir=f"model_output/selected_features_xgb/one_model/top{n_top}",
    ranked_filename = "lgb_rank_submission.parquet",
    raw_filename ="lgb_raw_submission.parquet",
)


✅ 已儲存原始 submission: model_output/selected_features_xgb/one_model/top20/lgb_raw_submission.parquet
shape: (6_897_776, 4)
┌──────────┬─────────────────────────────────┬───────────┬───────────────────┐
│ Id       ┆ ranker_id                       ┆ selected  ┆ __index_level_0__ │
│ ---      ┆ ---                             ┆ ---       ┆ ---               │
│ i64      ┆ str                             ┆ f64       ┆ i64               │
╞══════════╪═════════════════════════════════╪═══════════╪═══════════════════╡
│ 18144679 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 0.576973  ┆ 18144679          │
│ 18144680 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 0.500844  ┆ 18144680          │
│ 18144681 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ -0.2978   ┆ 18144681          │
│ 18144682 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 0.329973  ┆ 18144682          │
│ 18144683 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 0.429276  ┆ 18144683          │
│ …        ┆ …                               ┆ …         ┆ …                 │
│ 25043143 

: 

In [ ]:
from scripts.result_esemble import compute_ranked_average_selected_from_submissions

submission_paths = [
    "model_output/selected_features_xgb/one_model/top60/raw_submission.parquet",
    "model_output/all_features_xgb/2_raw_submission.parquet",
    "model_output/all_features_xgb/raw_submission.parquet",
    "model_output/selected_features_xgb/one_model/top20/raw_submission.parquet"
]

avg_df = compute_ranked_average_selected_from_submissions(submission_paths)

# 顯示前5筆
print(avg_df.head())

# 如需輸出
avg_df.write_parquet("submission_selected_mean.parquet")
print("✅ 已輸出平均submission")


✅ 讀取: model_output/selected_features_xgb/one_model/top60/raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/2_raw_submission.parquet
✅ 讀取: model_output/all_features_xgb/raw_submission.parquet
✅ 讀取: model_output/selected_features_xgb/one_model/top20/raw_submission.parquet
✅ 完成 6897776 筆平均+排名計算
shape: (5, 4)
┌──────────┬─────────────────────────────────┬──────────┬───────────────────┐
│ Id       ┆ ranker_id                       ┆ selected ┆ __index_level_0__ │
│ ---      ┆ ---                             ┆ ---      ┆ ---               │
│ i64      ┆ str                             ┆ u32      ┆ i64               │
╞══════════╪═════════════════════════════════╪══════════╪═══════════════════╡
│ 18144679 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 98       ┆ 18144679          │
│ 18144680 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 32       ┆ 18144680          │
│ 18144681 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 223      ┆ 18144681          │
│ 18144682 ┆ c9373e5f772e43d593dd6ad2fa90f6… ┆ 122      ┆ 18